# Simple queries against Spotify api to test OpenAPI-consuming agents

## API set up 

#### Spec

In [1]:
# https://github.com/APIs-guru/openapi-directory
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml

--2023-03-24 13:05:49--  https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286747 (280K) [text/plain]
Saving to: ‘openapi.yaml’

openapi.yaml        100%[===================>] 280.03K  --.-KB/s    in 0.05s   

2023-03-24 13:05:49 (5.36 MB/s) - ‘openapi.yaml’ saved [286747/286747]



In [2]:
import yaml
with open('openapi.yaml', 'r') as f:
    spotify_api_spec = yaml.load(f, Loader=yaml.Loader)

#### Authn/z
There's some setup to do:

1) we have to set up an app in Spotify's developer console by hand to populate these params.

https://developer.spotify.com/documentation/general/guides/authorization/app-settings/

In [3]:
CLIENT_ID = ''
CLIENT_SECRET = ''
REDIRECT_URI = 'http://thisisafakeaddress' # Using a fake address is fine, you just have to whitelist it in the Spotify developer console.

2) and we have to set up OAuth-style flows to get access + refresh tokens. 

A workaround is to install Spotipy and use that to get an access token (and refresh it).

In [5]:
# !pip install spotipy
import os
import spotipy.util as util

os.environ['SPOTIPY_CLIENT_ID'] = CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = CLIENT_SECRET
os.environ['SPOTIPY_REDIRECT_URI'] = REDIRECT_URI

auth_metadata = spotify_api_spec['components']['securitySchemes']['oauth_2_0']['flows']['authorizationCode']
scopes = list(auth_metadata['scopes'].keys())

access_token = util.prompt_for_user_token(scope=','.join(scopes))

In [6]:
headers = {
    'Authorization': f'Bearer {access_token}'
}

## Agent setup

For example, use the example existing json_explorer agent.

In [8]:
from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import RequestsWrapper
from langchain.tools.json.tool import JsonSpec

json_spec=JsonSpec(dict_=spotify_api_spec, max_value_length=4000)
requests_wrapper=RequestsWrapper(headers=headers)
openapi_toolkit = OpenAPIToolkit.from_llm(OpenAI(temperature=0), json_spec, requests_wrapper, verbose=True)

agent_executor = create_openapi_agent(
    llm=OpenAI(temperature=0),
    toolkit=openapi_toolkit,
    verbose=True,
    return_intermediate_steps=True
)

## Evaluate some simple queries

For context, asking some very simple queries about the track So What by Miles Davis:

- "What's the spotify id for the song 'So What'"
    - demonstrative, no user would ask for an api-internal id
    - can be done in a single api call
    - but has to be done via /search; other endpoints that seem relevant eg /tracks take the id as inpput
- "What's the track name for the song with id 4vLYewWIvqHfKtJDk8c8tq?"
    - same
- "What artist wrote the song So What"
    - also can be done in a single api call to /search
    - ofc, hallucination likely as So What is well-known
- "What are some other songs written by the artist who wrote So What"
    - requires several api calls
    - similarly hallucination prone

In [11]:
# Single-step: should call /search
# Demonstrative (no user would ask for a spotify id)
query = "What's the spotify id for the song 'So What'"
desired_output = '4sb0eMpDn3upAFfyi4q2rw'
out = agent_executor({'input': query})

# For another agent implementation..
# def _retrieves_search_endpoint(agent_out):
#     steps = agent_out['intermediate_steps']
#     for step in steps:
#         action, observation = step
#         if action.tool == 'api_endpoint_retriever':
#             if '/search' in observation:
#                 return True
#     return False

def _calls_search_endpoint(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'requests_get' and '/search' in action.tool_input:
            return True
    return False

def _check_answer(agent_out):
    output = agent_out['output']
    if desired_output in output:
        return True
    return False

tests = [_calls_search_endpoint, _check_answer]
for test_fn in tests:
    print(test_fn.__name__, test_fn(out))



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base url for the API?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'servers', 'info', 'paths', 'components']
Thought: I should look at the servers key to see what the base url is
Action: json_spec_list_keys
Action Input: data["servers"][0]
Observation: ValueError('Value at path `data["servers"][0]` is not a dict, get the value directly.')
Thought: I should get the value of the servers key
Action: json_spec_get_value
Action Input: data["servers"][0]
Observation: {'url': 'https://api.spotify.com/v1'}
Thought: I now know the base url for the API
Final Answer: The base url for the API is https://api.spotify.com/v1

> Finished chain.

Observation: The base url for the API is https://api.spotify.com/v1
Thought: I should find the path for getting a song's spotify id
Action: json_explorer
Action Input: What is the path for getting a song's sp

Thought: I should look for a path that contains tracks
Action: json_spec_list_keys
Action Input: data["paths"]["/tracks"]
Observation: ['get', 'x-spotify-docs-category', 'x-spotify-docs-display-name']
Thought: I now know the final answer
Final Answer: data["paths"]["/tracks"]

> Finished chain.

Observation: data["paths"]["/tracks"]
Thought: I should look for the parameters needed to make the request
Action: json_explorer
Action Input: What are the required parameters for a GET request to the /tracks endpoint?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'servers', 'info', 'paths', 'components']
Thought: I should look at the paths key to see what endpoints exist
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/albums', '/albums/{id}', '/albums/{id}/tracks', '/artists', '/artists/{id}', '/artists/{id}/albums', '/artists/{id}/related-artists', '/artists/{id}/top-tracks', '/audio-analysis/{id}', '/aud

In [ ]:
# Single-step: should call /track or /tracks
# Demonstrative (no user would have a spotify id)
query = "What's the track name for the song with id 4vLYewWIvqHfKtJDk8c8tq?"
out = agent_executor({'input': query})
desired_output = "So What"

# def _retrieves_tracks_endpoint(agent_out):
#     steps = agent_out['intermediate_steps']
#     for step in steps:
#         action, observation = step
#         if action.tool == 'api_endpoint_retriever':
#             if '/track' in observation:
#                 return True
#     return False

def _calls_tracks_endpoint(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'requests_get' and '/track' in action.tool_input:
            return True
    return False

def _check_answer(agent_out):
    final_obs = agent_out['intermediate_steps'][-1][1]
    output = agent_out['output']
    # Verify the answer comes from context...    
    if desired_output in final_obs and desired_output in output:
        return True
    return False

tests = [_calls_tracks_endpoint, _check_answer]
for test_fn in tests:
    print(test_fn.__name__, test_fn(out))

In [ ]:
# Single-step: calls /search
# Slightly harder
query = "What artist wrote the song So What"
out = agent_executor({'input': query})
desired_output = "Miles Davis"

# def _retrieves_search_endpoint(agent_out):
#     steps = agent_out['intermediate_steps']
#     for step in steps:
#         action, observation = step
#         if action.tool == 'api_endpoint_retriever':
#             if '/search' in observation:
#                 return True
#     return False

def _calls_search_endpoint(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'requests_get' and '/search' in action.tool_input:
            return True
    return False

def _check_answer(agent_out):
    final_obs = agent_out['intermediate_steps'][-1][1]
    output = agent_out['output']
    # Verify the answer comes from context...    
    if desired_output in final_obs and desired_output in output:
        return True
    return False

tests = [_uses_search_endpoint, _check_answer]
for test_fn in tests:
    print(test_fn.__name__, test_fn(out))

In [ ]:
# Multi-step: calls /search w/ song, then calls /search with artist or /artists/id/top-tracks
query = "What are some other songs written by the artist who wrote So What"
out = agent_executor({'input': query})

# def _retrieves_search_endpoint(agent_out):
#     steps = agent_out['intermediate_steps']
#     for step in steps:
#         action, observation = step
#         if action.tool == 'api_endpoint_retriever':
#             if '/search' in observation:
#                 return True
#     return False
    
def _calls_search_endpoint_for_song(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'requests_get' and '/search' in action.tool_input and 'So+What' in action.tool_input:
            return True
    return False

# ...

tests = [_calls_search_endpoint_for_song]
for test_fn in tests:
    print(test_fn.__name__, test_fn(out))